In [ ]:
!pip install qiskit==0.46 qiskit-ibmq-provider pylatexenc qiskit-aer

: 

In [ ]:
from qiskit import IBMQ, BasicAer
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
MY_API_TOKEN = 'eda22ec38f81a3f8bea973c9355c422d15824bb570d5044749f90c5a9d3f3ed7f5b15970fa374ef750355b4fe9402ec7f8f37cc7325920906f8c4f185e42f663'
IBMQ.save_account(MY_API_TOKEN)

IBMQ.load_account()

In [ ]:
n = 7
backend = Aer.get_backend('qasm_simulator')
prob_of_ans = []

In [5]:
target_value = '0111111'

In [ ]:
for x in range(15):
    database = QuantumRegister(n)
    oracle = QuantumRegister(1)
    auxiliary = QuantumRegister(n-1)
    cr = ClassicalRegister(n)
    qc = QuantumCircuit(database, oracle, auxiliary, cr)

    qc.h(database[:])
    qc.x(oracle[0])
    qc.h(oracle[0])

    for j in range(x):
        for k in range(n):
            if target_value[k] == '0':
                qc.x(database[k])
        qc.mct(database[:], oracle[0], auxiliary[:])
        for k in range(n):
            if target_value[k] == '0':
                qc.x(database[k])

        qc.h(database[:])
        qc.x(database[:])
        qc.h(database[n-1])
        qc.mct(database[:-1], database[n-1], auxiliary[:])
        qc.h(database[n-1])
        qc.x(database[:])
        qc.h(database[:])

    qc.h(oracle[0])
    qc.x(oracle[0])
    qc.measure(database, cr)
    qc = qc.reverse_bits()

    job = execute(qc, backend=backend, shots=1000)
    result = job.result()
    count = result.get_counts()

    answer = count.get(target_value, 0)
    prob_of_ans.append(answer)

In [ ]:
max_amplitude_iterations = np.argmax(prob_of_ans)
print(f'Количество итераций с наибольшей вероятностью: {max_amplitude_iterations}')

plt.figure(figsize=(10, 6))
plt.plot(range(15), prob_of_ans, marker='o')
plt.title('Вероятность нахождения целевого значения в зависимости от количества итераций')
plt.xlabel('Количество итераций')
plt.ylabel('Количество находок целевого значения')
plt.xticks(range(15))
plt.grid()
plt.show()